# Directory setup & Access

In [ ]:
%load_ext rpy2.ipython

In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from sklearn.metrics import roc_auc_score, accuracy_score, log_loss, precision_score, recall_score, f1_score

import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os, sys
import errno

# make a directory if it does not exist
def make_dir_if_not_exist(used_path):
    if not os.path.isdir(used_path):
        try:
            os.mkdir(used_path)
        except OSError as exc:
            if exc.errno != errno.EEXIST:
                raise exc
            else:
                raise ValueError(f'{used_path} directoy cannot be created because its parent directory does not exist.')

In [ ]:
paper_name = "veer"

time: 892 µs (started: 2022-04-06 14:04:41 +00:00)


In [ ]:
dataset = 'veer'

In [ ]:
# make directories if they do not exist
for paper_name_here in ["veer","boston"]:
  make_dir_if_not_exist("/content/drive/MyDrive/data_papers/")
  make_dir_if_not_exist(f"/content/drive/MyDrive/data_papers/{paper_name_here}")
  make_dir_if_not_exist(f"/content/drive/MyDrive/data_papers/{paper_name_here}/model_features/")
  make_dir_if_not_exist(f"/content/drive/MyDrive/data_papers/{paper_name_here}/concatenated_model_features/")
  make_dir_if_not_exist(f"/content/drive/MyDrive/data_papers/{paper_name_here}/model_checkpoints/")
  make_dir_if_not_exist(f"/content/drive/MyDrive/data_papers/{paper_name_here}/model_history/")
  make_dir_if_not_exist(f"/content/drive/MyDrive/data_papers/{paper_name_here}/model_finals/")
  make_dir_if_not_exist(f"/content/drive/MyDrive/data_papers/{paper_name_here}/gp_collab/")
  make_dir_if_not_exist(f"/content/drive/MyDrive/data_papers/{paper_name_here}/model_predictions/")
  make_dir_if_not_exist(f"/content/drive/MyDrive/data_papers/{paper_name_here}/model_ccs/")
  make_dir_if_not_exist(f"/content/drive/MyDrive/data_papers/{paper_name_here}/summary_results/")
  make_dir_if_not_exist(f"/content/drive/MyDrive/data_papers/{paper_name_here}/summary_results/temp/")


time: 27.3 ms (started: 2022-04-06 14:04:39 +00:00)


In [ ]:
%pip install ipython-autotime

In [ ]:
%load_ext autotime

time: 96.1 µs (started: 2022-04-06 12:09:32 +00:00)


In [ ]:
import tensorflow as tf
AUTOTUNE = tf.data.AUTOTUNE

time: 2.26 s (started: 2022-04-06 12:09:32 +00:00)


# GPFlow installation, result summary functions and kernel list construction

In [ ]:
%pip install gpflow

     |████████████████████████████████| 334 kB 14.3 MB/s 
     |████████████████████████████████| 104 kB 67.1 MB/s 
     |████████████████████████████████| 462 kB 77.7 MB/s 
time: 4.96 s (started: 2022-04-06 12:09:40 +00:00)


In [ ]:
import gpflow as gp

time: 1.26 s (started: 2022-04-06 12:09:45 +00:00)


In [ ]:
from gpflow import *

time: 859 µs (started: 2022-04-06 12:09:47 +00:00)


In [ ]:
def construct_kernel_list(num_of_independent_vars, base_lengthscales = [1.0]):
  possible_kernels = []
  for i in range(len(base_lengthscales)):
    possible_kernels.append(gp.kernels.Matern12(variance=1.0, lengthscales=[base_lengthscales[i]]*num_of_independent_vars))
    possible_kernels.append(gp.kernels.Matern32(variance=1.0, lengthscales=[base_lengthscales[i]]*num_of_independent_vars))
    possible_kernels.append(gp.kernels.RBF(variance=1.0, lengthscales=[base_lengthscales[i]]*num_of_independent_vars))
    possible_kernels.append(gp.kernels.Matern52(variance=1.0, lengthscales=[base_lengthscales[i]]*num_of_independent_vars))
    # possible_kernels.append(gpflow.kernels.SquaredExponential(lengthscales=[base_lengthscales[i]]*num_of_independent_vars))  
  return possible_kernels

time: 5.36 ms (started: 2022-04-06 12:09:47 +00:00)


In [ ]:
def construct_sum_of_kernels(num_of_independent_vars, base_lengthscales = [1.0]):
  kern_sum = [ gp.kernels.RBF(variance=1.0, lengthscales=base_lengthscales*num_of_independent_vars) + 
    gp.kernels.Matern12(variance=1.0, lengthscales=base_lengthscales*num_of_independent_vars) +
    gp.kernels.Matern32(variance=1.0, lengthscales=base_lengthscales*num_of_independent_vars) + 
    gp.kernels.Matern52(variance=1.0, lengthscales=base_lengthscales*num_of_independent_vars) + 
    gp.kernels.Linear(active_dims=[1]) +
    gp.kernels.Cosine(active_dims=[1]) ]
  
  kern_sum = kern_sum[0]
  kern_sum.kernels[0].variance.prior = tfp.distributions.Gamma(tf.cast(1.0, tf.float64), tf.cast(2.0, tf.float64))
  kern_sum.kernels[1].variance.prior = tfp.distributions.Gamma(tf.cast(1.0, tf.float64), tf.cast(2.0, tf.float64))
  kern_sum.kernels[2].variance.prior = tfp.distributions.Gamma(tf.cast(1.0, tf.float64), tf.cast(2.0, tf.float64))
  kern_sum.kernels[3].variance.prior = tfp.distributions.Gamma(tf.cast(1.0, tf.float64), tf.cast(2.0, tf.float64))
  kern_sum.kernels[4].variance.prior = tfp.distributions.Gamma(tf.cast(1.0, tf.float64), tf.cast(2.0, tf.float64))
  kern_sum.kernels[5].variance.prior = tfp.distributions.Gamma(tf.cast(1.0, tf.float64), tf.cast(2.0, tf.float64))

  return kern_sum


time: 11.1 ms (started: 2022-04-06 12:09:47 +00:00)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
 
def get_confusion_matrix_classification(model, X, Y_true):
    y_pred = model.predict(X)
    y_true = np.apply_along_axis(np.argmax, 1, Y_true)
    y_pred = np.apply_along_axis(np.argmax, 1, y_pred)
    return (confusion_matrix(y_true, y_pred), y_pred, y_true)

def construct_confusion_matrix(X, Y_true, Y_pred):
    y_true = Y_true
    y_pred = np.apply_along_axis(np.argmax, 1, Y_pred)
    return (confusion_matrix(y_true, y_pred), y_pred, y_true)

def pr_rc_f1_acc_from_supplied(y_pred, y_true):  
    pr, rc, f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")   
    acc = accuracy_score(y_true, y_pred)
    return pr, rc, f1, acc


time: 23.7 ms (started: 2022-04-06 12:09:47 +00:00)


In [ ]:
import tensorflow_probability as tfp

time: 852 µs (started: 2022-04-06 12:09:47 +00:00)


# Loading and using the training GP that was locally generated for Veer Data

In [ ]:
paper_name_here = "veer"

In [ ]:
gp_train  = pd.read_csv(f"/content/drive/MyDrive/data_papers/{paper_name_here}/veer_gp_train_data.csv",
                        sep=",")

time: 472 ms (started: 2022-04-06 12:36:00 +00:00)


In [ ]:
gp_test  = pd.read_csv(f"/content/drive/MyDrive/data_papers/{paper_name_here}/veer_gp_test_data.csv",
                        sep=",")

time: 365 ms (started: 2022-04-06 12:36:01 +00:00)


In [ ]:
gp_train

In [ ]:
train_targets = gp_train[["Y"]]
test_targets = gp_test[["Y"]]

train_data = gp_train[[x for x in gp_train.columns.values if x != "Y"]]
test_data = gp_test[[x for x in gp_test.columns.values if x != "Y"]]

train_data = train_data.to_numpy()
test_data = test_data.to_numpy()

train_targets = train_targets.to_numpy()
test_targets = test_targets.to_numpy()

time: 7.13 ms (started: 2022-04-06 12:40:07 +00:00)


In [ ]:
MAX_MIN = 1.5
import numpy as np

def ilogit(x):
  a = np.exp(x)
  return (a/(1.0+a))

def logit(x, max_min_val=MAX_MIN):
  return(np.clip(np.log(x/(1.0-x)),-max_min_val, max_min_val))


time: 2.47 ms (started: 2022-04-06 12:48:12 +00:00)


In [ ]:
train_data = logit(train_data)
test_data = logit(test_data)

time: 1.69 ms (started: 2022-04-06 12:49:23 +00:00)


/usr/local/lib/python3.7/dist-packages/gpflow/__init__.py:9: RuntimeWarning: divide by zero encountered in true_divide
  # Unless required by applicable law or agreed to in writing, software
/usr/local/lib/python3.7/dist-packages/gpflow/__init__.py:9: RuntimeWarning: divide by zero encountered in log
  # Unless required by applicable law or agreed to in writing, software


In [ ]:
np_y_train = train_targets
np_y_test = test_targets

np_x_train = train_data
np_x_test = test_data

time: 1.22 ms (started: 2022-04-06 12:49:34 +00:00)


In [ ]:
data_gp = (np_x_train, np_y_train)
data_gp_test = np_x_test
data_gp_test_target = np_y_test
id_str="test"
nth_inducing_ratio = 1.0
var_list = [1.0]       
early_stop_count = 100
early_stop_check_interval = 1000
early_stop_elbo_factor = 1.001
max_niters = 200000

time: 2.76 ms (started: 2022-04-06 12:49:36 +00:00)


In [ ]:
import random

num_of_independent_vars = data_gp[0].shape[1]
num_of_classes = np.unique(data_gp[1]).size
num_of_functions = num_of_classes
possible_kernels = construct_kernel_list(num_of_independent_vars, var_list)

# Robustmax Multiclass Likelihood
invlink = gp.likelihoods.RobustMax(num_of_functions)  # Robustmax inverse link function
likelihood = gp.likelihoods.MultiClass(num_of_functions, invlink=invlink)  # Multiclass likelihood
idxs_of_induced = sorted(random.sample(range(data_gp[0].shape[0]),int(data_gp[0].shape[0]/nth_inducing_ratio)))
inducing_inputs = data_gp[0][idxs_of_induced,:].copy()  # inducing inputs
gp_models = []
for kernel in possible_kernels:
  m = gp.models.SVGP(
      kernel=kernel,
      likelihood=likelihood,
      inducing_variable=inducing_inputs,
      num_latent_gps=num_of_functions,
      whiten=True,
      q_diag=True,
  )
  # set_trainable(m.inducing_variable, True)
  m.kernel.variance.prior = tfp.distributions.Gamma(tf.cast(0.5, tf.float64), tf.cast(1.0, tf.float64))
  gp_models.append(m)


time: 3.82 s (started: 2022-04-06 12:49:42 +00:00)


In [ ]:
result_dict = dict()
all_name = paper_name
df_to_return = None
idxCount = 0 

for mcount,m in enumerate(gp_models):

    print(mcount)
    tensor_data = tuple(map(tf.convert_to_tensor, data_gp))
    training_loss = m.training_loss_closure(tensor_data, compile=True)
    starting_elbo = -training_loss().numpy()
    print(f'Starting ELBO {starting_elbo}')
    elbos = [training_loss().numpy()]
    optimizer = tf.optimizers.Adam()  

    # optimizer = tf.optimizers.RMSprop()
    @tf.function
    def optim_here():
        optimizer.minimize(training_loss, m.trainable_variables)

    try:
      for itc in range(max_niters):
          optim_here()
          elbo_now = -training_loss().numpy()
          elbos.append(elbo_now)
          # print(elbo_now)
          if (itc % early_stop_check_interval) == 0:
            print(f'ELBO {elbo_now}')
            if len(elbos) > (early_stop_count+1):
              if elbos[-1]<0.0:
                if elbos[-early_stop_count] >= elbos[-1]*early_stop_elbo_factor:
                    print(f'Early stopping at {itc}')
                    break          
              else:
                if elbos[-early_stop_count] >= elbos[-1]*(2.0-early_stop_elbo_factor):
                    print(f'Early stopping at {itc}')
                    break

          # needs at least a decent improvement

      print(f'Ending ELBO {elbos[-1]}')

      y_pred_model, y_pred_model_V = m.predict_y(data_gp_test)  
      y_pred = np.apply_along_axis(np.argmax, 1, y_pred_model) 
      pr, rc, f1, acc =pr_rc_f1_acc_from_supplied(y_pred,data_gp_test_target)
      print (gp_models[mcount].kernel.name, mcount, pr, rc, f1, acc)

      if df_to_return is None:
        df_to_return = pd.DataFrame({ 
                "Core_Data" : all_name,
                "Type": "CNN_GPMETHOD2",
                "Data" : "Test",
                "NumOfModels": 1,
                "GPName": m.kernel.name,
                "KernelVariance": float(m.kernel.variance.numpy()),
                "nth_inducing_ratio": nth_inducing_ratio,
                "Pr": pr,
                "Rc": rc,
                "F1": f1,
                "Acc": acc
                 }, index = [idxCount])
      else:
        df_to_return = pd.concat([df_to_return, pd.DataFrame({
                "Core_Data" : all_name,
                "Type": "CNN_GPMETHOD2",
                "Data" : "Test",
                "NumOfModels": 1,
                "GPName": m.kernel.name,
                "KernelVariance": float(m.kernel.variance.numpy()),
                "nth_inducing_ratio": nth_inducing_ratio,
                "Pr": pr,
                "Rc": rc,
                "F1": f1,
                "Acc": acc
                }, index = [idxCount])])
      idxCount += 1
      
    except:
      print(f'ERROR, Ending ELBO {elbos[-1]} (NOT saved)')
      next


0
Starting ELBO -208.83503832239626
ELBO -208.32877447335707
ELBO -15.476549305699212
ELBO -12.853572023669692
ELBO -10.70917230515514
ELBO -8.41011983518257
ELBO -6.734421139424307
ELBO -5.606891893091379
ELBO -4.67939492932115
ELBO -3.8491365424227957
ELBO -3.0863879345173686
ELBO -2.3930022551190566
ELBO -1.795632441743086
ELBO -1.3483242810861311
ELBO -1.0851506040532213
ELBO -0.9428377665530201
ELBO -0.8252334001648247
ELBO -0.7331648534890931
ELBO -0.6513209832374178
ELBO -0.5783958899532475
ELBO -0.5131869107219282
ELBO -0.4546024573533529
ELBO -0.40150374989382875
ELBO -0.3536590714274981
ELBO -0.31003067664770345
ELBO -0.270254822187173
ELBO -0.22317990490813244
ELBO -0.19047673517439012
ELBO -0.1589484180217937
ELBO -0.13084415513343206
ELBO -0.10493432428653104
ELBO -0.08085316199893544
ELBO -0.05843728988056007
ELBO -0.037532150735211545
ELBO -0.01658747181675224
ELBO 0.002024985086958253
ELBO 0.02037277513339486
ELBO 0.03680544299614219
ELBO 0.0525275709484454
ELBO 0.06754

In [ ]:
df_to_return

,Core_Data,Type,Data,NumOfModels,GPName,KernelVariance,nth_inducing_ratio,Pr,Rc,F1,Acc
0,veer,CNN_GPMETHOD2,Test,1,matern12,0.000011,1.0,0.569444,0.555556,0.555556,0.555556
1,veer,CNN_GPMETHOD2,Test,1,matern32,0.000013,1.0,0.569444,0.555556,0.555556,0.555556
2,veer,CNN_GPMETHOD2,Test,1,squared_exponential,0.000013,1.0,0.569444,0.555556,0.555556,0.555556
3,veer,CNN_GPMETHOD2,Test,1,matern52,0.000012,1.0,0.569444,0.555556,0.555556,0.555556


time: 20.9 ms (started: 2022-04-06 14:01:41 +00:00)


In [ ]:

import datetime

df_to_return.to_csv( f"/content/drive/MyDrive/data_papers/{paper_name_here}/summary_results/GPMETHOD2_{all_name}_{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}.csv",
                     index = False ) 



time: 9.57 ms (started: 2022-04-06 14:02:07 +00:00)


In [ ]:
# in sample results

y_pred_model_IS, y_pred_model_V_IS = m.predict_y(data_gp[0])  
y_pred_IS = np.apply_along_axis(np.argmax, 1, y_pred_model_IS) 
pr, rc, f1, acc =pr_rc_f1_acc_from_supplied(y_pred_IS,data_gp[1])
print (m.kernel.name, mcount, pr, rc, f1, acc)


matern52 3 1.0 1.0 1.0 1.0
time: 54.9 ms (started: 2022-04-06 14:02:12 +00:00)


In [ ]:
paper_name_here

'veer'

time: 4.58 ms (started: 2022-04-06 14:03:40 +00:00)


# Loading and using the training GP that was locally generated for Boston Data

In [ ]:
paper_name_here = "boston"

time: 883 µs (started: 2022-04-06 15:45:55 +00:00)


In [ ]:
from sklearn.metrics import mean_squared_error
# rms = mean_squared_error(y_actual, y_predicted, squared=False)

time: 1.05 ms (started: 2022-04-06 15:45:56 +00:00)


In [ ]:
gp_train  = pd.read_csv(f"/content/drive/MyDrive/data_papers/{paper_name_here}/{paper_name_here}_gp_train_data.csv",
                        sep=",")

time: 10.6 ms (started: 2022-04-06 15:45:57 +00:00)


In [ ]:
gp_test  = pd.read_csv(f"/content/drive/MyDrive/data_papers/{paper_name_here}/{paper_name_here}_gp_test_data.csv",
                        sep=",")

time: 6.39 ms (started: 2022-04-06 15:49:50 +00:00)


In [ ]:
gp_test.head(5)

,SL.xgboost_All,SL.randomForest_All,SL.glmnet_0_All,SL.glmnet_0.25_All,SL.glmnet_0.5_All,SL.glmnet_0.75_All,SL.glmnet_1_All,SL.ksvm_All,SL.biglasso_All,SL.lm_All,SL.mean_All,Y
0,26.530006,26.793378,25.843590,25.760138,25.797661,25.805099,25.674092,24.347419,25.731151,25.612917,22.860396,28.7
1,22.724335,21.493117,23.113421,22.917559,22.931376,22.930248,22.882279,20.020488,22.894258,22.868513,22.860396,22.9
2,17.663872,20.427108,11.848391,10.838068,10.834711,10.816860,10.678199,18.575654,10.720084,10.634706,22.860396,16.5
3,21.253941,21.378820,19.376044,18.768071,18.800329,18.794514,18.632181,18.955167,18.678779,18.579445,22.860396,18.9
4,22.876255,22.041046,19.550344,18.826838,18.852631,18.843036,18.671324,19.900210,18.720717,18.615861,22.860396,15.0


time: 17.7 ms (started: 2022-04-06 15:49:53 +00:00)


In [ ]:
def get_numpy_from_df(dt, ycol="Y"):
  df = dt.copy()
  targets = df[[ycol]].to_numpy()
  data = df[[x for x in df.columns.values if x != ycol]].to_numpy()
  return (data, targets)


np_x_train, np_y_train = get_numpy_from_df(gp_train,"Y")
np_x_test, np_y_test = get_numpy_from_df(gp_test,"Y")


time: 7.15 ms (started: 2022-04-06 15:50:43 +00:00)


In [ ]:
data_gp = (np_x_train, np_y_train)
data_gp_test = np_x_test
data_gp_test_target = np_y_test
id_str="test"
var_list = [1.0]       

time: 2.18 ms (started: 2022-04-06 15:50:53 +00:00)


In [ ]:
import random

num_of_independent_vars = data_gp[0].shape[1]
possible_kernels = construct_kernel_list(num_of_independent_vars, var_list)

# data_gp = tuple(map(tf.convert_to_tensor, data_gp))

gp_models = []
for kernel in possible_kernels:
  m = gp.models.GPR(data=(np_x_train, np_y_train), 
                    kernel=kernel, 
                    mean_function=None)
  # set_trainable(m.inducing_variable, True)
  m.kernel.variance.prior = tfp.distributions.Gamma(tf.cast(0.5, tf.float64), tf.cast(1.0, tf.float64))
  gp_models.append(m)


time: 104 ms (started: 2022-04-06 15:51:07 +00:00)


In [ ]:
result_dict = dict()
all_name = paper_name_here
df_to_return = None
idxCount = 0 

for mcount,m in enumerate(gp_models):

    print(mcount)
    opt = gp.optimizers.Scipy()
    opt_logs = opt.minimize(m.training_loss, m.trainable_variables, options=dict(maxiter=100))
    y_pred_model, y_pred_model_V = m.predict_y(data_gp_test) 

    mse = mean_squared_error(np_y_test, y_pred_model, squared=True) 
    print (gp_models[mcount].kernel.name, mse)

    if df_to_return is None:
      df_to_return = pd.DataFrame({ 
              "Core_Data" : all_name,
              "Type": "CNN_GPMETHOD2",
              "Data" : "Test",
              "NumOfModels": 1,
              "GPName": m.kernel.name,
              "KernelVariance": float(m.kernel.variance.numpy()),
              "MSE": mse
                }, index = [idxCount])
    else:
      df_to_return = pd.concat([df_to_return, pd.DataFrame({
              "Core_Data" : all_name,
              "Type": "CNN_GPMETHOD2",
              "Data" : "Test",
              "NumOfModels": 1,
              "GPName": m.kernel.name,
              "KernelVariance": float(m.kernel.variance.numpy()),
              "MSE": mse
              }, index = [idxCount])])
    idxCount += 1

0
matern12 62.816410060032396
1
matern32 17.314295717936965
2
squared_exponential 19.494192419090528
3
matern52 19.474951735023865
time: 3.35 s (started: 2022-04-06 15:51:12 +00:00)


In [ ]:
boston_base_train = pd.read_csv(f"/content/drive/MyDrive/data_papers/{paper_name_here}/{paper_name_here}_base_train.csv")
boston_base_test = pd.read_csv(f"/content/drive/MyDrive/data_papers/{paper_name_here}/{paper_name_here}_base_test.csv")

time: 13.1 ms (started: 2022-04-06 15:51:25 +00:00)


In [ ]:
base_train, base_train_targets = get_numpy_from_df(boston_base_train,"medv")
base_test, base_test_targets = get_numpy_from_df(boston_base_test,"medv")

time: 5.28 ms (started: 2022-04-06 15:58:03 +00:00)


In [ ]:
print(base_train.shape)
print(base_train_targets.shape)

(404, 13)
(404, 1)
time: 1.6 ms (started: 2022-04-06 15:58:03 +00:00)


In [ ]:
construct_kernel_list

<function gpflow.construct_kernel_list>

time: 3.28 ms (started: 2022-04-06 15:58:25 +00:00)


In [ ]:
possible_kernels = construct_kernel_list(base_train.shape[1], [1.0])
gp_models_base = []

# base_data_gp = tuple(map(tf.convert_to_tensor, (base_train,base_train_targets)))

for kernel in possible_kernels:
  m = gp.models.GPR(data=base_data_gp, 
                    kernel=kernel, 
                    mean_function=None)
  # set_trainable(m.inducing_variable, True)
  m.kernel.variance.prior = tfp.distributions.Gamma(tf.cast(0.5, tf.float64), tf.cast(1.0, tf.float64))
  gp_models_base.append(m)


time: 110 ms (started: 2022-04-06 15:59:04 +00:00)


In [ ]:
base_train[0,:]

array([6.320e-03, 1.800e+01, 2.310e+00, 0.000e+00, 5.380e-01, 6.575e+00,
       6.520e+01, 4.090e+00, 1.000e+00, 2.960e+02, 1.530e+01, 3.969e+02,
       4.980e+00])

time: 4.11 ms (started: 2022-04-06 15:51:38 +00:00)


In [ ]:
for mcount,m in enumerate(gp_models_base):
  
    print(mcount)
    opt = gp.optimizers.Scipy()
    opt_logs = opt.minimize(m.training_loss, m.trainable_variables, options=dict(maxiter=100))
    y_pred_model, y_pred_model_V = m.predict_y(base_test) 

    mse = mean_squared_error(base_test_targets, y_pred_model, squared=True) 
    print (m.kernel.name, mse)


0
matern12 12.782087794110993
1
matern32 9.017795086177232
2
squared_exponential 11.242421795614998
3
matern52 12.982249585998636
time: 4.11 s (started: 2022-04-06 15:59:09 +00:00)


In [ ]:
import datetime
df_to_return.to_csv( f"/content/drive/MyDrive/data_papers/{paper_name_here}/summary_results/GPMETHOD2_{all_name}_{datetime.datetime.now().strftime('%Y%m%d%H%M%S')}.csv",
                     index = False ) 

In [ ]:
# in sample results

y_pred_model_IS, y_pred_model_V_IS = m.predict_y(data_gp[0])  
y_pred_IS = np.apply_along_axis(np.argmax, 1, y_pred_model_IS) 
pr, rc, f1, acc =pr_rc_f1_acc_from_supplied(y_pred_IS,data_gp[1])
print (m.kernel.name, mcount, pr, rc, f1, acc)


In [ ]:
paper_name_here

# R code used for the setup - locally

In [ ]:
# %%R


# # install.packages(c("Depends","Imports","tidyverse","foreach","data.table"), quietly=T)
# # install.packages(c("glmnet","randomForest","class","gam","gbm","nnet","polspline","MASS","e1071","stepPlr","arm","party","spls","LogicReg"), quietly=T)
# # install.packages(c("nnls","multicore","SIS","BayesTree","quadprog","ipred","mlbench","rpart","caret","mda","earth"), quietly=T)
# # install.packages("dsa", quietly=T)
# # if (!require("BiocManager", quietly = TRUE))
# #   install.packages("BiocManager", quietly=T)
# # BiocManager::install(version = "3.14", quietly=T)
# # BiocManager::install(c("Biobase"), quietly=T) # "GenomicFeatures", "AnnotationDbi"
# # install.packages("SuperLearner")
# # install.packages(c("xgboost","KernelKnn","bartMachine"))
# # install.packages("rJava")
# # install.packages("randomForestSRC")
# # BiocManager::install(c("breastCancerNKI"), quietly=T)
# # BiocManager::install(c("Biobase"), quietly=T)
# # install.packages("biglasso")
# # install.packages("mltools")

# # sl_lib = c("SL.xgboost", "SL.randomForest", "SL.glmnet", "SL.ksvm",
# #            "SL.bayesglm", "SL.caret", "SL.cforest", "SL.gam",
# #            "SL.logreg", "SL.biglasso",
# #            "SL.bartMachine", "SL.kernelKnn", "SL.rpartPrune", "SL.lm", "SL.mean")

# # head(vdv[vdv$Censoring!=0,1:10],4)
# # head(vdv[,1:10],4)

# require(SuperLearner)
# require(biglasso)
# require(mltools)

# # data(Boston, package = "MASS")
# data(vdv, package = "randomForestSRC")


# set.seed(1)
# train_idxs = sort(sample(1:nrow(vdv),60))
# vdv_train = vdv[train_idxs,]
# vdv_test = vdv[-train_idxs,]


# # "SL.nnet"
# listWrappers()

# # Fit elastic net with 5 different alphas: 0, 0.2, 0.4, 0.6, 0.8, 1.0.
# # 0 corresponds to ridge and 1 to lasso.
# enet = create.Learner("SL.glmnet", detailed_names = T,
#                       tune = list(alpha = seq(0, 1, length.out = 5))) # enet$names,


# # sl_lib = c("SL.xgboost", "SL.randomForest", enet$names, 
# #             "SL.gam",  "SL.glm", "SL.bartMachine", "SL.lm", "SL.mean")
# # result_all = SuperLearner(Y = vdv$Censoring, X = vdv[, 3:ncol(vdv)], SL.library = sl_lib)
# # sl_xgboost = SuperLearner(Y = vdv$Censoring, X = vdv[, 3:ncol(vdv)], SL.library = "SL.xgboost")
# # sl_randomForest = SuperLearner(Y = vdv$Censoring, X = vdv[, 3:ncol(vdv)],SL.library = "SL.randomForest")

# sl_lib_binomial_a = c("SL.xgboost", "SL.randomForest", enet$names, "SL.lm", "SL.mean") # works
# sl_lib_binomial_b = c("SL.xgboost", "SL.randomForest", enet$names, "SL.ksvm", "SL.biglasso", "SL.lm", "SL.mean") # works

# # sl_lib_binomial = c("SL.xgboost", "SL.randomForest", enet$names, "SL.ksvm", "SL.biglasso", "SL.lm", "SL.mean") 
# # result_all_binomial = SuperLearner(Y = vdv$Censoring, X = vdv[, 3:ncol(vdv)], 
# #                                    family=binomial(), SL.library = sl_lib_binomial)
# # sl_xgboost_binomial = SuperLearner(Y = vdv$Censoring, X = vdv[, 3:ncol(vdv)], family=binomial(), SL.library = "SL.xgboost")
# # sl_randomForest_binomial = SuperLearner(Y = vdv$Censoring, X = vdv[, 3:ncol(vdv)], family=binomial(), SL.library = "SL.randomForest")
# # sl_glmnet0_binomial = SuperLearner(Y = vdv$Censoring, X = vdv[, 3:ncol(vdv)], family=binomial(), SL.library = enet$names[1])
# # sl_glmnet0p25_binomial = SuperLearner(Y = vdv$Censoring, X = vdv[, 3:ncol(vdv)], family=binomial(), SL.library = enet$names[2])
# # sl_glmnet0p5_binomial = SuperLearner(Y = vdv$Censoring, X = vdv[, 3:ncol(vdv)], family=binomial(), SL.library = enet$names[3])
# # sl_glmnet0p75_binomial = SuperLearner(Y = vdv$Censoring, X = vdv[, 3:ncol(vdv)], family=binomial(), SL.library = enet$names[4])
# # sl_glmnet1_binomial = SuperLearner(Y = vdv$Censoring, X = vdv[, 3:ncol(vdv)], family=binomial(), SL.library = enet$names[5])
# # sl_ksvm_binomial = SuperLearner(Y = vdv$Censoring, X = vdv[, 3:ncol(vdv)], family=binomial(), SL.library = "SL.ksvm")
# # sl_bilasso_binomial = SuperLearner(Y = vdv$Censoring, X = vdv[, 3:ncol(vdv)], family=binomial(), SL.library = "SL.biglasso")
# # sl_lm_binomial = SuperLearner(Y = vdv$Censoring, X = vdv[, 3:ncol(vdv)], family=binomial(), SL.library = "SL.lm")

# sl_lib_binomial = c("SL.xgboost", "SL.randomForest", enet$names, "SL.ksvm", "SL.biglasso", "SL.lm", "SL.mean") 
# result_all_binomial_train = SuperLearner(Y = vdv_train$Censoring, X = vdv_train[, 3:ncol(vdv_train)], 
#                                    family=binomial(), SL.library = sl_lib_binomial)

# sl_xgboost_binomial_train = SuperLearner(Y = vdv_train$Censoring, X = vdv_train[, 3:ncol(vdv_train)], family=binomial(), SL.library = "SL.xgboost")
# sl_randomForest_binomial_train = SuperLearner(Y = vdv_train$Censoring, X = vdv_train[, 3:ncol(vdv_train)], family=binomial(), SL.library = "SL.randomForest")
# sl_glmnet0_binomial_train = SuperLearner(Y = vdv_train$Censoring, X = vdv_train[, 3:ncol(vdv_train)], family=binomial(), SL.library = enet$names[1])
# sl_glmnet0p25_binomial_train = SuperLearner(Y = vdv_train$Censoring, X = vdv_train[, 3:ncol(vdv_train)], family=binomial(), SL.library = enet$names[2])
# sl_glmnet0p5_binomial_train = SuperLearner(Y = vdv_train$Censoring, X = vdv_train[, 3:ncol(vdv_train)], family=binomial(), SL.library = enet$names[3])
# sl_glmnet0p75_binomial_train = SuperLearner(Y = vdv_train$Censoring, X = vdv_train[, 3:ncol(vdv_train)], family=binomial(), SL.library = enet$names[4])
# sl_glmnet1_binomial_train = SuperLearner(Y = vdv_train$Censoring, X = vdv_train[, 3:ncol(vdv_train)], family=binomial(), SL.library = enet$names[5])
# sl_ksvm_binomial_train = SuperLearner(Y = vdv_train$Censoring, X = vdv_train[, 3:ncol(vdv_train)], family=binomial(), SL.library = "SL.ksvm")
# sl_bilasso_binomial_train = SuperLearner(Y = vdv_train$Censoring, X = vdv_train[, 3:ncol(vdv_train)], family=binomial(), SL.library = "SL.biglasso")
# sl_lm_binomial_train = SuperLearner(Y = vdv_train$Censoring, X = vdv_train[, 3:ncol(vdv_train)], family=binomial(), SL.library = "SL.lm")

# pred_xgboost_test = predict(sl_xgboost_binomial_train, vdv_test[, 3:ncol(vdv_test)])
# pred_randomForest_test = predict(sl_randomForest_binomial_train, vdv_test[, 3:ncol(vdv_test)])
# pred_rglmnet0_test = predict(sl_glmnet0_binomial_train, vdv_test[, 3:ncol(vdv_test)])
# pred_rglmnet0p25_test = predict(sl_glmnet0p25_binomial_train, vdv_test[, 3:ncol(vdv_test)])
# pred_rglmnet0p5_test = predict(sl_glmnet0p5_binomial_train, vdv_test[, 3:ncol(vdv_test)])
# pred_rglmnet0p75_test = predict(sl_glmnet0p75_binomial_train, vdv_test[, 3:ncol(vdv_test)])
# pred_rglmnet1_test = predict(sl_glmnet1_binomial_train, vdv_test[, 3:ncol(vdv_test)])
# pred_ksvm_test = predict(sl_ksvm_binomial_train, vdv_test[, 3:ncol(vdv_test)])
# pred_biglasso_test = predict(sl_bilasso_binomial_train, vdv_test[, 3:ncol(vdv_test)])
# pred_lm_test = predict(sl_lm_binomial_train, vdv_test[, 3:ncol(vdv_test)])

# pred_SL_all_binomial_test = predict(result_all_binomial_train, vdv_test[, 3:ncol(vdv_test)])

# vdv_train$Censoring
# vdv_test$Censoring

# require(mltools)
# auc_roc(as.vector(pred_SL_all_binomial_test$pred), vdv_test$Censoring)
# auc_roc(as.vector(pred_xgboost_test$pred), vdv_test$Censoring)
# auc_roc(as.vector(pred_randomForest_test$pred), vdv_test$Censoring)
# auc_roc(as.vector(pred_rglmnet0_test$pred), vdv_test$Censoring)
# auc_roc(as.vector(pred_rglmnet0p25_test$pred), vdv_test$Censoring)
# auc_roc(as.vector(pred_rglmnet0p5_test$pred), vdv_test$Censoring)
# auc_roc(as.vector(pred_rglmnet0p75_test$pred), vdv_test$Censoring)
# auc_roc(as.vector(pred_rglmnet1_test$pred), vdv_test$Censoring)
# auc_roc(as.vector(pred_ksvm_test$pred), vdv_test$Censoring)
# auc_roc(as.vector(pred_biglasso_test$pred), vdv_test$Censoring)
# auc_roc(as.vector(pred_lm_test$pred), vdv_test$Censoring)

# rmse_test <- function(pred_res, act_res) {
#   return( sqrt(mean((as.vector(pred_res$pred)-as.vector(act_res$Censoring))^2))  )
# }

# rmse_test(pred_SL_all_binomial_test, vdv_test)
# rmse_test(pred_xgboost_test, vdv_test)
# rmse_test(pred_randomForest_test, vdv_test)
# rmse_test(pred_rglmnet0_test, vdv_test)
# rmse_test(pred_rglmnet0p25_test, vdv_test)
# rmse_test(pred_rglmnet0p5_test, vdv_test)
# rmse_test(pred_rglmnet0p75_test, vdv_test)
# rmse_test(pred_rglmnet1_test, vdv_test)
# rmse_test(pred_ksvm_test, vdv_test)
# rmse_test(pred_biglasso_test, vdv_test)
# rmse_test(pred_lm_test, vdv_test)

# ########### GP inputs

# pred_xgboost_train = predict(sl_xgboost_binomial_train, vdv_train[, 3:ncol(vdv_test)])
# pred_randomForest_train = predict(sl_randomForest_binomial_train, vdv_train[, 3:ncol(vdv_test)])
# pred_rglmnet0_train = predict(sl_glmnet0_binomial_train, vdv_train[, 3:ncol(vdv_test)])
# pred_rglmnet0p25_train = predict(sl_glmnet0p25_binomial_train, vdv_train[, 3:ncol(vdv_test)])
# pred_rglmnet0p5_train = predict(sl_glmnet0p5_binomial_train, vdv_train[, 3:ncol(vdv_test)])
# pred_rglmnet0p75_train = predict(sl_glmnet0p75_binomial_train, vdv_train[, 3:ncol(vdv_test)])
# pred_rglmnet1_train = predict(sl_glmnet1_binomial_train, vdv_train[, 3:ncol(vdv_test)])
# pred_ksvm_train = predict(sl_ksvm_binomial_train, vdv_train[, 3:ncol(vdv_test)])
# pred_biglasso_train = predict(sl_bilasso_binomial_train, vdv_train[, 3:ncol(vdv_test)])
# pred_lm_train = predict(sl_lm_binomial_train, vdv_train[, 3:ncol(vdv_test)])


# gp_train_data = data.frame(cbind(pred_xgboost_train$library.predict, 
#       pred_randomForest_train$library.predict,
#       pred_rglmnet0_train$library.predict,
#       pred_rglmnet0p25_train$library.predict,
#       pred_rglmnet0p5_train$library.predict,
#       pred_rglmnet0p75_train$library.predict,
#       pred_rglmnet1_train$library.predict,
#       pred_ksvm_train$library.predict,
#       pred_biglasso_train$library.predict,
#       pred_lm_train$library.predict))

# require(data.table)
# fwrite(gp_train_data, "/Users/hansroggeman/Code/gp_train_data.csv")

# # result = SuperLearner(Y = Boston$medv, X = Boston[, -14], SL.library = sl_lib)
# # result_binom = SuperLearner(Y = vdv$Censoring, X = vdv[, 3:ncol(vdv)], 
# #                             family = binomial(),
# #                             SL.library = sl_lib)
# # Review performance of each algorithm and ensemble weights.
# result
# # Use external (aka nested) cross-validation to estimate ensemble accuracy.
# # This will take a while to run.
# result2 = CV.SuperLearner(Y = Boston$medv, X = Boston[, -14], SL.library = sl_lib)
# # Plot performance of individual algorithms and compare to the ensemble.
# require(ggthemes)
# plot(result2) # + theme_minimal()
# # Hyperparameter optimization --
# sl_lib2 = c("SL.mean", "SL.lm", enet$names)
# enet_sl = SuperLearner(Y = Boston$medv, X = Boston[, -14], SL.library = sl_lib2)
# # Identify the best-performing alpha value or use the automatic ensemble.
# enet_sl
# enet_sl2 = CV.SuperLearner(Y = Boston$medv, X = Boston[, -14], SL.library = sl_lib2)

# plot(enet_sl2)





In [ ]:
# %%R


# require(SuperLearner)

# data(Boston, package = "MASS")

# set.seed(1)

# sl_lib = c("SL.xgboost", "SL.randomForest", "SL.glmnet", "SL.nnet", "SL.ksvm",
#            "SL.bartMachine", "SL.kernelKnn", "SL.rpartPrune", "SL.lm", "SL.mean")

# # Fit XGBoost, RF, Lasso, Neural Net, SVM, BART, K-nearest neighbors, Decision Tree, 
# # OLS, and simple mean; create automatic ensemble.
# result = SuperLearner(Y = Boston$medv, X = Boston[, -14], SL.library = sl_lib)
# # Review performance of each algorithm and ensemble weights.
# result

# # Use external (aka nested) cross-validation to estimate ensemble accuracy.
# # This will take a while to run.
# result2 = CV.SuperLearner(Y = Boston$medv, X = Boston[, -14], SL.library = sl_lib)

# # Plot performance of individual algorithms and compare to the ensemble.
# plot(result2) + theme_minimal()

# # Hyperparameter optimization --
# # Fit elastic net with 5 different alphas: 0, 0.2, 0.4, 0.6, 0.8, 1.0.
# # 0 corresponds to ridge and 1 to lasso.
# enet = create.Learner("SL.glmnet", detailed_names = T,
#                       tune = list(alpha = seq(0, 1, length.out = 5)))

# sl_lib2 = c("SL.mean", "SL.lm", enet$names)

# enet_sl = SuperLearner(Y = Boston$medv, X = Boston[, -14], SL.library = sl_lib2)

# # Identify the best-performing alpha value or use the automatic ensemble.
# enet_sl

In [ ]:
# %%R

# install.packages(c("ggplot2","ggthemes"), quietly=T)